In [55]:
using CSV
using DataFrames
using XLSX
using Statistics
using Glob
using MLJ
using MLJBase
using DelimitedFiles
using CategoricalArrays

In [68]:
response_name = ["no_tuber_scab", "no_tuber_scabpit"
    , "no_tuber_scabsuper", "yield_per_meter", "yield_per_plant", "pctg_black_scurf"]
for i in 1:6
    write_to_non_level(i, response_name[i])
end

In [67]:
function write_to_non_level(response, name)
    data = get_result(response)
    filename = "../processed-data/result_file/result_table/disease_suppression/$name.xlsx"
    labels = collect(0:1:5)
    XLSX.openxlsx(filename, mode="w") do xf
        sheet = xf[1]
        XLSX.writetable!(sheet, data, vcat("", labels))
    end
end

write_to_non_level (generic function with 1 method)

In [66]:
function get_result(response)
    path = "../processed-data/result_file/non_augumented/suppression/"
    filename = glob("*$response.txt", path)
    data = single_response(filename)
    return data
end

get_result (generic function with 1 method)

In [56]:
function write_to_xlsx(response, name)
    data = get_all_level(response)
    filename = "../processed-data/result_file/result_table/alpha_diversity_indices/$name.xlsx"
    labels = collect(0:1:6)
    XLSX.openxlsx(filename, mode="w") do xf
        sheet = xf[1]
        XLSX.rename!(sheet, "Phylum")
        XLSX.writetable!(sheet, data[1], vcat("", labels))
        XLSX.addsheet!(xf)
        XLSX.rename!(xf[2], "Class")
        XLSX.writetable!(xf[2], data[2], vcat("", labels))
        XLSX.addsheet!(xf)
        XLSX.rename!(xf[3], "Order")
        XLSX.writetable!(xf[3], data[3], vcat("", labels))
        XLSX.addsheet!(xf)
        XLSX.rename!(xf[4], "Family")
        XLSX.writetable!(xf[4], data[4], vcat("", labels))
        XLSX.addsheet!(xf)
        XLSX.rename!(xf[5], "Genus")
        XLSX.writetable!(xf[5], data[5], vcat("", labels))
    end
end

write_to_xlsx (generic function with 1 method)

In [57]:
function get_all_level(response)
    all_level = ["Phylum", "Class", "Order","Family", "Genus"]
    all = Any[]
    for i in 1:length(all_level)
        level = all_level[i]
        path = "../processed-data/result_file/non_augumented/alpha/$level/"
        filename = glob("*$response.txt", path)
        data = single_response(filename)
        push!(all, data)
    end    
    return all
end

get_all_level (generic function with 1 method)

In [58]:
function single_response(filename)
    all = Any[]
    row_num = [0,1,2,3,4]
    push!(all, row_num)
    for i in 1:length(filename)
        data = readdlm(filename[i])
        new = calculation(data)
        push!(all, new)
    end
    return all
end

single_response (generic function with 1 method)

In [59]:
function calculation(data)
    ret = zeros(5)
    # set prediction and truth
    truth = categorical(data[:, 2])
    predict = categorical(data[:, 4])
    measures("FScore")
    # F for zero
    m = FScore(rev=false)
    ret[1] = m(predict, truth)
    if isnan(ret[1])
        ret[1] = 0
    end
    # F for one
    m = FScore(rev=true)
    ret[2] = m(predict, truth)
    if isnan(ret[2])
        ret[2] = 0
    end    
    ret[3] = accuracy(predict, truth)
    m = MulticlassFScore(average = MLJBase.MacroAvg())
    ret[4] = m(predict, truth)
    p = [(i, count(==(i), truth)) for i in unique(truth)]
    if p[1][2] == 0
        zero = p[1][2]
        one = p[2][2]
    else
        zero = p[2][2]
        one = p[1][2]
    end
    #println(zero, ", ", one)
    ret[5] = (zero*ret[1] + one*ret[2])/(zero + one)
    
    return ret
end

calculation (generic function with 1 method)